In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D,Dropout,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
#chargement de dataset à partir de google drive
from google.colab import drive
drive.mount('/content/drive')

import os
print(os.listdir("/content/drive/MyDrive/DATASET"))

train_data="/content/drive/MyDrive/DATASET/TRAIN"
test_data="/content/drive/MyDrive/DATASET/TEST"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['TRAIN', 'TEST']


In [4]:
#créer le modèle
# On utilise un modèle séquentiel : empilement linéaire de couches
model=Sequential()
# === Bloc 1 : Extraction de caractéristiques simples (bords, textures)
# - 32 filtres de taille 3x3 : détectent les motifs élémentaires.
# - Activation ReLU : introduit la non-linéarité (met à zéro les valeurs négatives).
# - input_shape : dimension de l’image d’entrée (ici 224x224 RGB).
model.add(Conv2D(32,kernel_size=(3,3),activation="relu",input_shape=(224,224,3)))
# - MaxPooling2D : réduit la dimension spatiale (hauteur et largeur ÷2)
#   tout en gardant les informations principales (valeur max).
model.add(MaxPooling2D(pool_size=(2,2)))

# === Bloc 2 : Extraction de caractéristiques intermédiaires
# - 64 filtres : le réseau apprend des motifs plus complexes (formes locales).
# - Même logique de convolution + pooling.
model.add(Conv2D(64,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

# === Bloc 3 : Extraction de caractéristiques avancées
# - 128 filtres : capture des structures plus abstraites ou globales.
# - Plus la profondeur augmente, plus les features deviennent conceptuelles.
model.add(Conv2D(128,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

# === Bloc 4 : Passage de la représentation 2D à 1D
# - Flatten : transforme les cartes de caractéristiques (H×W×C) en un vecteur 1D.
model.add(Flatten())
# === Bloc 5 : Couches Fully Connected (denses)
# - 128 neurones : combinent les features extraites pour la décision finale.
# - Activation ReLU : accélère l’apprentissage et améliore la performance.
model.add(Dense(128,activation="relu"))
# - Dropout(0.5) : désactive aléatoirement 50 % des neurones à chaque étape
#   pour éviter le surapprentissage (régularisation).
model.add(Dropout(0.5))
# === Bloc 6 : Couche de sortie
# - 1 neurone avec activation sigmoid : adapté à une classification binaire (0 ou 1)
#   Exemple : plastique vs métal, recyclable vs non recyclable.
model.add(Dense(1,activation="sigmoid"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


###  le rôle des blocs convolutionnels du CNN

Le modèle CNN est constitué de plusieurs blocs `Conv2D` et `MaxPooling2D` qui travaillent ensemble pour extraire progressivement des informations de plus en plus abstraites à partir de l’image.  
Chaque bloc joue un rôle précis :

- **Bloc 1 — 32 filtres :**  
  Ce premier bloc agit comme une “loupe de bas niveau”.  
  Il détecte les **motifs simples** tels que les bords, les lignes et les contrastes locaux.  
  Ces caractéristiques servent de base à tous les motifs plus complexes des couches suivantes.

- **Bloc 2 — 64 filtres :**  
  En combinant les motifs élémentaires détectés précédemment, ce bloc apprend à reconnaître des **formes locales** et des **textures plus complexes**.  
  Doubler le nombre de filtres permet au modèle de capturer un plus grand nombre de variations visuelles.

- **Bloc 3 — 128 filtres :**  
  À ce stade, le réseau identifie des **parties d’objets** (par exemple, une roue, un coin, un bord caractéristique d’un matériau).  
  Les filtres deviennent plus nombreux pour modéliser la richesse des informations visuelles présentes dans les images.

- **Bloc 4 — 256 filtres (facultatif selon la taille de l’image) :**  
  Ce bloc capture une **compréhension globale** des objets présents dans l’image.  
  Il combine plusieurs caractéristiques locales pour reconnaître des **formes complètes ou des concepts visuels entiers** (ex. : plastique, métal, verre).

- **Bloc Dense final :**  
  Les cartes de caractéristiques issues des couches convolutionnelles sont aplaties (`Flatten`) et transmises à une couche dense (`Dense`).  
  Cette dernière combine toutes les informations extraites pour **prendre la décision finale de classification**.

>  En résumé :  
> Au fur et à mesure que l’on progresse dans les blocs (32 → 64 → 128 → 256),  
> la taille spatiale des images diminue (à cause du `Pooling`),  
> mais la **profondeur augmente**, ce qui permet au réseau de représenter des motifs de plus en plus abstraits et significatifs.


In [5]:
#compiler le modèle
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [6]:
train_data_gen=ImageDataGenerator(rescale=1./255)
test_data_gen=ImageDataGenerator(rescale=1./255)

In [7]:
train_gen=train_data_gen.flow_from_directory(
    train_data,
    target_size=(224,224),
    batch_size=128,
    color_mode="rgb",
    class_mode="binary",
)

Found 22578 images belonging to 2 classes.


In [8]:
test_gen=test_data_gen.flow_from_directory(
    test_data,
    target_size=(224,224),
    batch_size=32,
    color_mode="rgb",
    class_mode="binary",
)

Found 2513 images belonging to 2 classes.


In [ ]:
history=model.fit(train_gen,epochs=10,validation_data=test_gen)
import matplotlib.pyplot as plt

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("Loss")
plt.legend()
plt.show()
model.save("/content/drive/MyDrive/model.h5")

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
  2/177 ━━━━━━━━━━━━━━━━━━━━ 3:21:23 69s/step - accuracy: 0.5234 - loss: 1.8432